In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import sqlite3
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from prophet import Prophet
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error, median_absolute_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler


In [2]:
immigration_data_2012_2021 = "Resources/immigration_data_2005_2021.csv"
#state_unemployment = "Resources/emp-unemployment.xls"
#gini = "Resources/gini_index_by_state_2016_2018.xlsx"

In [3]:
inmigration = pd.read_csv(immigration_data_2012_2021)
inmigration.head()

,Year,Region and country of birth,Total Permanent Residents,Percentage,Alabama,Alaska,Arizona,Arkansas,California,Colorado,...,Virginia,Washington,West Virginia,Wisconsin,Wyoming,U.S. Armed Services posts,U.S. possessions,U.S. Dependencies,Guam,Puerto Rico
0,2005,Total,1122373,100.00,4200,1525,18988,2698,232023,11977,...,27100,26482,847,7909,321,128,5868,0,0,0
1,2005,"China, People's Republic",69967,6.23,328,92,543,202,17668,765,...,1327,1508,101,593,28,13,184,0,0,0
2,2005,Dominican Republic,27504,2.45,5,42,22,0,82,6,...,90,18,6,39,0,0,2346,0,0,0
3,2005,India,84681,7.54,431,15,739,215,14724,516,...,2776,1747,133,876,0,0,24,0,0,0
4,2005,Iran,13887,1.24,48,4,285,9,7059,131,...,562,318,18,48,0,0,0,0,0,0


## Cleaning country regressor dataset

In [4]:
#Read the csv
countries_raw_metadata = pd.read_csv("Resources/countries_metadata.csv", encoding='latin-1')
display(countries_raw_metadata.head())
print(countries_raw_metadata.shape)

,Country Name,Country Code,Series Name,Series Code,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],...,2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022]
0,China,CHN,"Population, total",SP.POP.TOTL,667070000,660330000,665770000,682335000,698355000,715185000,...,1363240000,1371860000,1379860000,1387790000,1396215000,1.402760e+09,1407745000,1411100000,1412360000,1412175000
1,China,CHN,GDP per capita (current US$),NY.GDP.PCAP.CD,89.5202179159242,75.8055639067957,70.9091553472599,74.3133748233246,85.4982461036748,98.4864217470782,...,7020.38568208449,7636.07422276004,8016.44601585644,8094.39037512163,8817.04549566316,9.905406e+03,10143.8602060373,10408.7191247747,12617.5049863004,12720.2156397612
2,China,CHN,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.NE.ZS,..,..,..,..,..,..,...,4.05,4.1,..,..,3.9,4.930000e+00,5.15,5.61,5.11,..
3,China,CHN,Urban population (% of total population),SP.URB.TOTL.IN.ZS,16.203,16.708,17.226,17.757,18.299,18.086,...,53.013,54.259,55.5,56.736,57.96,5.915200e+01,60.308,61.428,62.512,63.56
4,China,CHN,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,20.86,18.02,37.01,43.37,39.14,37.88,...,13.03,13.83,11.99,13.57,12.64,1.086000e+01,10.41,8.52,7.52,..


(70, 67)


In [5]:
#Delete the "[YR..]" in the year columns
for column in countries_raw_metadata.columns:
    if "[YR" in column:
        new_column_name = column.split("[")[0].strip() # Extract the part before '[YR' and remove any leading/trailing spaces
        countries_raw_metadata.rename(columns={column: new_column_name}, inplace=True)
countries_raw_metadata.head()

,Country Name,Country Code,Series Name,Series Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,China,CHN,"Population, total",SP.POP.TOTL,667070000,660330000,665770000,682335000,698355000,715185000,...,1363240000,1371860000,1379860000,1387790000,1396215000,1.402760e+09,1407745000,1411100000,1412360000,1412175000
1,China,CHN,GDP per capita (current US$),NY.GDP.PCAP.CD,89.5202179159242,75.8055639067957,70.9091553472599,74.3133748233246,85.4982461036748,98.4864217470782,...,7020.38568208449,7636.07422276004,8016.44601585644,8094.39037512163,8817.04549566316,9.905406e+03,10143.8602060373,10408.7191247747,12617.5049863004,12720.2156397612
2,China,CHN,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.NE.ZS,..,..,..,..,..,..,...,4.05,4.1,..,..,3.9,4.930000e+00,5.15,5.61,5.11,..
3,China,CHN,Urban population (% of total population),SP.URB.TOTL.IN.ZS,16.203,16.708,17.226,17.757,18.299,18.086,...,53.013,54.259,55.5,56.736,57.96,5.915200e+01,60.308,61.428,62.512,63.56
4,China,CHN,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,20.86,18.02,37.01,43.37,39.14,37.88,...,13.03,13.83,11.99,13.57,12.64,1.086000e+01,10.41,8.52,7.52,..


In [6]:
#Clean the "Country Names" columns 
unique_names = countries_raw_metadata["Country Name"].unique()
unique_names

array(['China', 'United States', 'Iran, Islamic Rep.', 'India',
       'Dominican Republic', 'Mexico', 'United Kingdom', 'Pakistan',
       'Philippines', nan,
       'Data from database: World Development Indicators',
       'Last Updated: 06/29/2023', 'Code', 'SP.POP.TOTL',
       'NY.GDP.PCAP.CD', 'SL.UEM.TOTL.NE.ZS',
       'Unemployment is a key measure to monitor whether a country is on track to achieve the Sustainable Development Goal of promoting sustained',
       'However', 'SP.URB.TOTL.IN.ZS',
       "Percentages urban are the numbers of persons residing in an area defined as ''urban'' per 100 total population. They are calculated by the Statistics Division of the United Nations Department of Economic and Social Affairs. Particular caution should be used in interpreting the figures for percentage urban for different countries.",
       'Countries differ in the way they classify population as "urban" or "rural." The population of a city or metropolitan area depends on the bou

In [7]:
#List the country names we want to keep.
attributes_to_keep = ['China', 'United States', 'Iran, Islamic Rep.', 'India', 'Dominican Republic', 'Mexico', 'United Kingdom', 'Pakistan', 'Philippines']

# Filter the DataFrame to keep only the desired attributes
df_filtered = countries_raw_metadata[countries_raw_metadata['Country Name'].isin(attributes_to_keep)]

# Print the filtered DataFrame
df_filtered.head()

,Country Name,Country Code,Series Name,Series Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,China,CHN,"Population, total",SP.POP.TOTL,667070000,660330000,665770000,682335000,698355000,715185000,...,1363240000,1371860000,1379860000,1387790000,1396215000,1.402760e+09,1407745000,1411100000,1412360000,1412175000
1,China,CHN,GDP per capita (current US$),NY.GDP.PCAP.CD,89.5202179159242,75.8055639067957,70.9091553472599,74.3133748233246,85.4982461036748,98.4864217470782,...,7020.38568208449,7636.07422276004,8016.44601585644,8094.39037512163,8817.04549566316,9.905406e+03,10143.8602060373,10408.7191247747,12617.5049863004,12720.2156397612
2,China,CHN,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.NE.ZS,..,..,..,..,..,..,...,4.05,4.1,..,..,3.9,4.930000e+00,5.15,5.61,5.11,..
3,China,CHN,Urban population (% of total population),SP.URB.TOTL.IN.ZS,16.203,16.708,17.226,17.757,18.299,18.086,...,53.013,54.259,55.5,56.736,57.96,5.915200e+01,60.308,61.428,62.512,63.56
4,China,CHN,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,20.86,18.02,37.01,43.37,39.14,37.88,...,13.03,13.83,11.99,13.57,12.64,1.086000e+01,10.41,8.52,7.52,..


In [8]:
#Prepare the "Series Name" to move it to the columns, these will be our regressors. 
series_names = df_filtered["Series Name"].unique()
series_names

array(['Population, total', 'GDP per capita (current US$)',
       'Unemployment, total (% of total labor force) (national estimate)',
       'Urban population (% of total population)',
       'Birth rate, crude (per 1,000 people)', 'Gini index'], dtype=object)

In [9]:
df_filtered.loc[df_filtered["Series Name"] == 'Population, total', "Series Name"] = "Population_total"
df_filtered.loc[df_filtered["Series Name"] == 'GDP per capita (current US$)', "Series Name"] = "GDP_per_capita"
df_filtered.loc[df_filtered["Series Name"] == 'Unemployment, total (% of total labor force) (national estimate)', "Series Name"] = "UR"
df_filtered.loc[df_filtered["Series Name"] == 'Urban population (% of total population)', "Series Name"] = "Urban_population"
df_filtered.loc[df_filtered["Series Name"] == 'Birth rate, crude (per 1,000 people)', "Series Name"] = "Birth_rate_crude"
df_filtered.loc[df_filtered["Series Name"] == 'Gini index', "Series Name"] = "Gini_index"
df_filtered.loc[df_filtered["Country Name"] == 'Iran, Islamic Rep.', "Country Name"] = "Iran"
df_filtered

,Country Name,Country Code,Series Name,Series Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,China,CHN,Population_total,SP.POP.TOTL,667070000,660330000,665770000,682335000,698355000,715185000,...,1363240000,1371860000,1379860000,1387790000,1396215000,1.402760e+09,1407745000,1411100000,1412360000,1412175000
1,China,CHN,GDP_per_capita,NY.GDP.PCAP.CD,89.5202179159242,75.8055639067957,70.9091553472599,74.3133748233246,85.4982461036748,98.4864217470782,...,7020.38568208449,7636.07422276004,8016.44601585644,8094.39037512163,8817.04549566316,9.905406e+03,10143.8602060373,10408.7191247747,12617.5049863004,12720.2156397612
2,China,CHN,UR,SL.UEM.TOTL.NE.ZS,..,..,..,..,..,..,...,4.05,4.1,..,..,3.9,4.930000e+00,5.15,5.61,5.11,..
3,China,CHN,Urban_population,SP.URB.TOTL.IN.ZS,16.203,16.708,17.226,17.757,18.299,18.086,...,53.013,54.259,55.5,56.736,57.96,5.915200e+01,60.308,61.428,62.512,63.56
4,China,CHN,Birth_rate_crude,SP.DYN.CBRT.IN,20.86,18.02,37.01,43.37,39.14,37.88,...,13.03,13.83,11.99,13.57,12.64,1.086000e+01,10.41,8.52,7.52,..
5,China,CHN,Gini_index,SI.POV.GINI,..,..,..,..,..,..,...,39.7,39.2,38.6,38.5,39.1,3.850000e+01,38.2,..,..,..
6,United States,USA,Population_total,SP.POP.TOTL,180671000,183691000,186538000,189242000,191889000,194303000,...,316059947,318386329,320738994,323071755,325122128,3.268382e+08,328329953,331511512,332031554,333287557
7,United States,USA,GDP_per_capita,NY.GDP.PCAP.CD,3007.12344537862,3066.56286916615,3243.84307754988,3374.51517105082,3573.94118474743,3827.52710972039,...,53291.1276891406,55123.8497869046,56762.7294515989,57866.7449341091,59907.754260885,6.282331e+04,65120.3946628653,63528.6343027508,70219.472454115,76398.5917422054
8,United States,USA,UR,SL.UEM.TOTL.NE.ZS,5.5,6.7,5.5,5.7,5.2,4.5,...,7.37,6.17,5.28,4.87,4.36,3.900000e+00,3.67,8.05,5.35,3.65
9,United States,USA,Urban_population,SP.URB.TOTL.IN.ZS,69.996,70.377,70.757,71.134,71.508,71.879,...,81.299,81.483,81.671,81.862,82.058,8.225600e+01,82.459,82.664,82.873,83.084


In [10]:
# Get rid the unnecessary columns
countries_regressors_raw = df_filtered.drop(columns= {"Country Code", "Series Code"} )
countries_regressors_raw.head()

,Country Name,Series Name,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,China,Population_total,667070000,660330000,665770000,682335000,698355000,715185000,735400000,754550000,...,1363240000,1371860000,1379860000,1387790000,1396215000,1.402760e+09,1407745000,1411100000,1412360000,1412175000
1,China,GDP_per_capita,89.5202179159242,75.8055639067957,70.9091553472599,74.3133748233246,85.4982461036748,98.4864217470782,104.324189073855,96.58918279471,...,7020.38568208449,7636.07422276004,8016.44601585644,8094.39037512163,8817.04549566316,9.905406e+03,10143.8602060373,10408.7191247747,12617.5049863004,12720.2156397612
2,China,UR,..,..,..,..,..,..,..,..,...,4.05,4.1,..,..,3.9,4.930000e+00,5.15,5.61,5.11,..
3,China,Urban_population,16.203,16.708,17.226,17.757,18.299,18.086,17.915,17.785,...,53.013,54.259,55.5,56.736,57.96,5.915200e+01,60.308,61.428,62.512,63.56
4,China,Birth_rate_crude,20.86,18.02,37.01,43.37,39.14,37.88,35.05,33.96,...,13.03,13.83,11.99,13.57,12.64,1.086000e+01,10.41,8.52,7.52,..


In [11]:
#countries_regressors_raw.info()

### Cleaning US state regressors

In [12]:
variables_by_state_raw = pd.read_csv("Resources/variable 2011-2021.csv")
variables_by_state_rename = variables_by_state_raw.rename(columns={"Unemployeement Rate": "UR_STATE",
                                                                  "GDP Per Capita":"GDP_per_capita_state", "Birth Rate":"Bith_rate_state"  })

# Multiply the "unemployment rate" column by 10
variables_by_state_rename["UR_STATE"] *= 10

## Cleaning inmigration csv

In [13]:
migration = inmigration.rename(columns={"Region and country of birth": "Country_origin", "Total Permanent Residents": "Total_inmigrants"})
migration = migration[migration["Country_origin"]!= "Total"]
migration = migration.drop(columns="Percentage")
migration["Country_origin"] = migration["Country_origin"].replace("China, People's Republic", "China")
migration

,Year,Country_origin,Total_inmigrants,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,...,Virginia,Washington,West Virginia,Wisconsin,Wyoming,U.S. Armed Services posts,U.S. possessions,U.S. Dependencies,Guam,Puerto Rico
1,2005,China,69967,328,92,543,202,17668,765,894,...,1327,1508,101,593,28,13,184,0,0,0
2,2005,Dominican Republic,27504,5,42,22,0,82,6,319,...,90,18,6,39,0,0,2346,0,0,0
3,2005,India,84681,431,15,739,215,14724,516,1571,...,2776,1747,133,876,0,0,24,0,0,0
4,2005,Iran,13887,48,4,285,9,7059,131,88,...,562,318,18,48,0,0,0,0,0,0
5,2005,Mexico,161445,569,96,8373,870,63092,2891,316,...,870,2330,30,1054,75,0,129,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,2021,Iran,5734,21,0,86,17,2206,89,39,...,218,187,9,38,0,0,0,0,0,0
149,2021,Mexico,107230,323,33,6859,610,31715,3131,247,...,586,2539,20,949,69,0,0,0,0,39
150,2021,Pakistan,9691,31,0,59,41,1104,52,111,...,784,166,17,57,3,0,0,0,0,0
151,2021,Philippines,27511,152,190,555,129,6478,228,178,...,648,737,69,236,27,0,0,0,300,9


### Create a Database and store the cleaned dataset

In [14]:
# Create database with the desired file name for your SQLite database
conn = sqlite3.connect('use_migration.db')

# Store DatFrames in the database
migration.to_sql('migration', conn, index=False,  if_exists='replace')
countries_regressors_raw.to_sql('countries_regressors_raw', conn, index=False,  if_exists='replace')
variables_by_state_rename.to_sql('variables_by_state_rename', conn, index=False,  if_exists='replace')
conn.close()

In [15]:
## Check if the DataFrames have been successfully stored in the SQLite database

# Connect to the SQLite database
conn = sqlite3.connect('use_migration.db')

# Read the data from the database
query = "SELECT * FROM migration;"
df_migration = pd.read_sql_query(query, conn)

query = "SELECT * FROM countries_regressors_raw;"
df_countries_regressors_raw = pd.read_sql_query(query, conn)

query = "SELECT * FROM variables_by_state_rename;"
df_variables_by_state_rename = pd.read_sql_query(query, conn)

# Check the data
#print("Data from migration:")
#print(df_migration)

#print("\nData from countries_regressors_raw:")
#print(df_countries_regressors_raw)

#print("\nData from variables_by_state_rename:")
#print(df_variables_by_state_rename)

# Close the database connection
#conn.close()



In [20]:
country_names=['China', 'India',
       'Dominican Republic', 'Mexico', 'United Kingdom', 'Pakistan',
       'Philippines']


state_names=['Alabama', 'Alaska',
       'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut',
       'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii',
       'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
       'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan',
       'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming']


results_df = pd.DataFrame(columns=['Country', 'State', 'MAE', 'MAPE','pred2023'])
# DataFrame to store predictions
df_predictions = pd.DataFrame()
df_scores = pd.DataFrame()
# Collect all predictions and true values for final R-squared calculation
all_preds = []
all_true = []

for country in country_names:
    
    # Filter by country
    regressor_by_country = df_countries_regressors_raw.loc[countries_regressors_raw["Country Name"]== country]
    # Drop the Country Name column
    drop_countryname = regressor_by_country.drop(columns="Country Name")
    # Transpose rows per columns
    regressor_by_country_not_cleaned = drop_countryname.transpose()
    # Use the first row as the column names
    regressor_by_country_not_cleaned.columns = regressor_by_country_not_cleaned.iloc[0]
    # Drop the first row (previous integer index) since it's now redundant
    regressor_by_country_drop_row = regressor_by_country_not_cleaned.iloc[1:]
    #print(country)
    regressor_by_country_reset_index = regressor_by_country_drop_row.reset_index().rename(columns={"index":"Year"})
    #display(regressor_by_country_reset_index)
    regressor_by_country_reset_index["Year"] = regressor_by_country_reset_index["Year"].astype(int)
    regressor_by_country_reset_index["Population_total"] = regressor_by_country_reset_index["Population_total"].astype(int)
    regressor_by_country_reset_index["GDP_per_capita"] = regressor_by_country_reset_index["GDP_per_capita"].astype(float)
    regressor_by_country_reset_index['UR'].replace('..', np.nan, inplace=True)
    regressor_by_country_reset_index["UR"] = regressor_by_country_reset_index["UR"].astype(float)
    regressor_by_country_reset_index["Urban_population"] = regressor_by_country_reset_index["Urban_population"].astype(float)
    regressor_by_country_reset_index['Birth_rate_crude'].replace('..', np.nan, inplace=True)
    regressor_by_country_reset_index["Birth_rate_crude"] = regressor_by_country_reset_index["Birth_rate_crude"].astype(float)
    regressor_by_country_reset_index['Gini_index'].replace('..', np.nan, inplace=True)
    regressor_by_country_reset_index["Gini_index"] = regressor_by_country_reset_index["Gini_index"].astype(float)
    regressor_by_country = regressor_by_country_reset_index
    
    
    for state in state_names:



        # Getting the DataFrame with the years, total number of inmigrants in the US for a given country and 
        # the number of inmigrants of this country in one US state
        migration.reset_index(drop=True, inplace=True)
        county_origin_us_df = df_migration[migration["Country_origin"]==country]
        country_columns = ["Year","Total_inmigrants",state]
        inmigration_us_df = county_origin_us_df[country_columns]

        #Filter by state
        state_test = df_variables_by_state_rename[variables_by_state_rename["State"]==state]
        regressor_by_state = state_test.drop(columns= "State").reset_index(drop=True)
    
        # First merge inmigration_us_df and regressor_by_country
        df_merged1 = pd.merge(inmigration_us_df, regressor_by_country, on="Year", how="left")

        # Then merge the result with regressor_by_state
        df_merged = pd.merge(df_merged1, regressor_by_state, on="Year", how="left")
        def set_covid(row):
            if row["Year"] in [2020, 2021]:
                return 1
            else:
                return 0

        # Add the Covid column using the apply method
        df_merged["Covid"] = df_merged.apply(set_covid, axis=1)

        print(df_merged)
        # LINEAR REGRESSION
        
        # Handle NaN values in the DataFrame
        # For simplicity, we'll fill the NaN values with the column means
        df_merged.fillna(df_merged.mean(), inplace=True)
        
        # Prepare the state-specific data
        cols_to_scale = ['Year', 'Population_total', 'GDP_per_capita', 'Urban_population',
                 'Birth_rate_crude', 'Gini_index', 
                 'GDP_per_capita_state', 'UR_STATE']
        X = df_merged[cols_to_scale]  # Features
        y = df_merged[state].values  # Target for the specific state
        
        
        # Create a min-max scaler object and apply normalization
        scaler = MinMaxScaler()
        X = scaler.fit_transform(X)
        

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Create and train the model
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Calculate predicted values
        y_pred = model.predict(X_test)
        
        # Append predicted and true values to the respective lists
        all_preds.extend(y_pred)
        all_true.extend(y_test)

        # Calculate R2 score, RMSE, and MAPE
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-10)))   # add a small constant to avoid division by zero
        

        # Store scores in the scores DataFrame
        df_scores = df_scores.append({
            'Country': country,
            'State': state,
            'R2 Score': r2,
            'RMSE': rmse,
            'MAPE': mape
        }, ignore_index=True)

        # Create a list of next years
        next_years = pd.DataFrame({
            'Year': np.array(range(2024, 2026)),
            'Population_total': np.nan,  # replace with actual values for next years if available
            'GDP_per_capita': np.nan,  # replace with actual values for next years if available
            'Urban_population': np.nan,  # replace with actual values for next years if available
            'Birth_rate_crude': np.nan,  # replace with actual values for next years if available
            'Gini_index': np.nan,  # replace with actual values for next years if available
            'GDP_per_capita_state': np.nan,  # replace with actual values for next years if available
            'UR_STATE': np.nan  # replace with actual values for next years if available
        })
        
        # Perform linear interpolation for each column in next_years
        for col in next_years.columns:
            next_years[col] = next_years[col].interpolate(method='linear')

        # Normalize next_years using the same scaler (handle NaN values)
        next_years_scaled = scaler.transform(next_years[cols_to_scale].fillna(df_merged.mean()))
        
        
        
        # Predict the immigration for the next years and store in the predictions DataFrame
        predictions = model.predict(next_years_scaled)
        for i, year in enumerate(next_years['Year'].values):
            df_predictions = df_predictions.append({
                'Year': year,
                'Country': country,
                'State': state,
                'Predicted Immigration': predictions[i]
            }, ignore_index=True)
    
    
        

        
        


    Year  Total_inmigrants  Alabama  Population_total  GDP_per_capita    UR  \
0   2005             69967      328        1303720000     1753.414263  4.20   
1   2006             87345      301        1311020000     2099.219342  4.10   
2   2007             76655      259        1317885000     2693.958555  4.00   
3   2008             80271      300        1324655000     3468.327063  4.20   
4   2009             64238      240        1331260000     3832.227428  4.30   
5   2010             70863      315        1337705000     4550.473924  4.10   
6   2011             87016      317        1345035000     5614.385739  4.10   
7   2012             81784      299        1354190000     6300.582352  4.10   
8   2013             71798      312        1363240000     7020.385682  4.05   
9   2014             76089      257        1371860000     7636.074223  4.10   
10  2015             74558      245        1379860000     8016.446016   NaN   
11  2016             81772      330        138779000

C:\Users\MIRIAM\AppData\Local\Temp\ipykernel_10340\1056533046.py:125: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\MIRIAM\AppData\Local\Temp\ipykernel_10340\1056533046.py:157: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\MIRIAM\AppData\Local\Temp\ipykernel_10340\1056533046.py:157: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\MIRIAM\AppData\Local\Temp\ipykernel_10340\1056533046.py:125: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\MIRIAM\AppData\Local\Temp\i

    Year  Total_inmigrants  District of Columbia  Population_total  \
0   2005             69967                   113        1303720000   
1   2006             87345                   133        1311020000   
2   2007             76655                    85        1317885000   
3   2008             80271                    84        1324655000   
4   2009             64238                    85        1331260000   
5   2010             70863                   115        1337705000   
6   2011             87016                   122        1345035000   
7   2012             81784                   126        1354190000   
8   2013             71798                   207        1363240000   
9   2014             76089                   396        1371860000   
10  2015             74558                   198        1379860000   
11  2016             81772                   209        1387790000   
12  2017             71565                   160        1396215000   
13  2018            

C:\Users\MIRIAM\AppData\Local\Temp\ipykernel_10340\1056533046.py:125: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\MIRIAM\AppData\Local\Temp\ipykernel_10340\1056533046.py:157: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\MIRIAM\AppData\Local\Temp\ipykernel_10340\1056533046.py:157: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\MIRIAM\AppData\Local\Temp\ipykernel_10340\1056533046.py:125: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\MIRIAM\AppData\Local\Temp\i

    Year  Total_inmigrants  Louisiana  Population_total  GDP_per_capita    UR  \
0   2005             69967        285        1303720000     1753.414263  4.20   
1   2006             87345        182        1311020000     2099.219342  4.10   
2   2007             76655        248        1317885000     2693.958555  4.00   
3   2008             80271        240        1324655000     3468.327063  4.20   
4   2009             64238        228        1331260000     3832.227428  4.30   
5   2010             70863        324        1337705000     4550.473924  4.10   
6   2011             87016        337        1345035000     5614.385739  4.10   
7   2012             81784        349        1354190000     6300.582352  4.10   
8   2013             71798        253        1363240000     7020.385682  4.05   
9   2014             76089        267        1371860000     7636.074223  4.10   
10  2015             74558        310        1379860000     8016.446016   NaN   
11  2016             81772  

...

C:\Users\MIRIAM\AppData\Local\Temp\ipykernel_10340\1056533046.py:125: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\MIRIAM\AppData\Local\Temp\ipykernel_10340\1056533046.py:157: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\MIRIAM\AppData\Local\Temp\ipykernel_10340\1056533046.py:157: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\MIRIAM\AppData\Local\Temp\ipykernel_10340\1056533046.py:125: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\MIRIAM\AppData\Local\Temp\i

    Year  Total_inmigrants  Pennsylvania  Population_total  GDP_per_capita  \
0   2005             60748           844          86261250     1245.286591   
1   2006             74607           865          87901835     1452.221420   
2   2007             72596           676          89561377     1741.603505   
3   2008             54030           537          91252326     1990.356146   
4   2009             60029           544          92946951     1893.281623   
5   2010             58173           582          94636700     2201.776819   
6   2011             57011           584          96337913     2431.199965   
7   2012             57327           555          98032317     2671.777538   
8   2013             54446           585          99700107     2847.567954   
9   2014             49996           477         101325201     2935.928578   
10  2015             56478           590         103031365     2974.296920   
11  2016             53287           606         104875266     3

    Year  Total_inmigrants  Vermont  Population_total  GDP_per_capita    UR  \
0   2005             60748       16          86261250     1245.286591  3.80   
1   2006             74607       35          87901835     1452.221420  4.05   
2   2007             72596       43          89561377     1741.603505  3.43   
3   2008             54030       23          91252326     1990.356146  3.72   
4   2009             60029       26          92946951     1893.281623  3.86   
5   2010             58173       29          94636700     2201.776819  3.61   
6   2011             57011       26          96337913     2431.199965  3.59   
7   2012             57327       28          98032317     2671.777538  3.50   
8   2013             54446       25          99700107     2847.567954  3.50   
9   2014             49996       29         101325201     2935.928578  3.60   
10  2015             56478       15         103031365     2974.296920  3.07   
11  2016             53287       15         10487526

C:\Users\MIRIAM\AppData\Local\Temp\ipykernel_10340\1056533046.py:157: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\MIRIAM\AppData\Local\Temp\ipykernel_10340\1056533046.py:157: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\MIRIAM\AppData\Local\Temp\ipykernel_10340\1056533046.py:125: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append({
C:\Users\MIRIAM\AppData\Local\Temp\ipykernel_10340\1056533046.py:157: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predictions = df_predictions.append({
C:\Users\MIRIAM\AppData\Lo

SyntaxError: 'break' outside loop (1056533046.py, line 166)

In [19]:
df_predictions

,Year,Country,State,Predicted Immigration
0,2024,China,Alabama,252.315908
1,2025,China,Alabama,251.941833
2,2024,China,Alaska,-340.606324
3,2025,China,Alaska,-376.580943
4,2024,China,Arizona,9319.023359
...,...,...,...,...
709,2025,Philippines,West Virginia,-595.633236
710,2024,Philippines,Wisconsin,3297.211738
711,2025,Philippines,Wisconsin,3571.096715
712,2024,Philippines,Wyoming,808.401907


In [17]:

df_predictions.to_csv("Output/Linear_Regression_predictions.csv", index=False)

In [18]:
# Close the database connection
conn.close()